# OM Duet Issuer - Test Notebook

### This notebook represents a Duet Issuer, that accepts connections from other agents and automatically issues them with a credential. In the future this will be deployed on a cloud.

Run through 1-8 which initialises the agent, registers a DID on the Sovrin test network and publishes a credential definition for a specific credential.

Part 8. creates a multi use invitation which should be copied into the relevant section in the Data Scientist notebook.

## 1. Initialise a controller for Issuer Agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
import time
from termcolor import colored,cprint

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8032
ADMIN_URL = "http://0.0.0.0:8031"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [2]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': '5zaPrJvh2JaRx59rBPTS8S', 'verkey': '3iq4WnafwfLPBQ1MK85Hw18DomEr1rbL6VcRkcNse7mc', 'posture': 'wallet_only'}


## 3. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [3]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "5zaPrJvh2JaRx59rBPTS8S": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by 5zaPrJvh2JaRx59rBPTS8S to the ledger with role ENDORSER"}}'}
200


In [4]:
response = await agent_controller.ledger.get_taa()
TAA = response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
print(TAA)

{'text': '\ufeff# Transaction Author Agreement V2\nhttps://sovrin.org/\n\n\n## Summary:\n\n\nThis summary is provided to help you understand your obligations when writing to\nthe Sovrin Ledger Networks-it does not have any legal effect or replace the full\nlegal text of the agreement provided below it.\n\n\n- This agreement grants you permission to write data to the Sovrin Ledger\n  Networks under certain terms and conditions.\n\n\n- You represent and warrant that the data you are writing does not violate any\n  applicable laws or infringe the rights of any other party.\n\n\n- You understand the data you are writing is public and permanent and there can\n  be no guarantee of erasure. This includes public keys and payment addresses.\n\n\n- If it is determined that the data you wrote violated this agreement, the\n  operators of the network can take steps to block it from public access.\n\n\n- The Sovrin Foundation makes no promises about the reliability or correctness\n  of the data bein

In [5]:
response = await agent_controller.ledger.accept_taa(TAA)
## Will return {} if successful
print(response)

{}


## 4. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [6]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': '5zaPrJvh2JaRx59rBPTS8S', 'verkey': '3iq4WnafwfLPBQ1MK85Hw18DomEr1rbL6VcRkcNse7mc', 'posture': 'public'}}


## 5. Write a Credential Definition to the Ledger

The schema already exists on the Sovrin staging network. 

You can view the related transactions, including the one you are just about to write, [here](https://indyscan.io/txs/SOVRIN_STAGINGNET/domain?page=1&pageSize=50&filterTxNames=[]&sortFromRecent=true&search=D4zNtr48UJy7MdbsupTKbF:2:SSI%20Duet%20Tutorial:0.0.1).

In [7]:

schema_id = "D4zNtr48UJy7MdbsupTKbF:2:SSI Duet Tutorial:0.0.1"
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]

## 6. Populate the Attribues to Issue to Idenity Holder (User)

In [8]:
credential_attributes = [
    {"name": "connection_permitted", "value": "1"}
]
print(credential_attributes)

[{'name': 'connection_permitted', 'value': '1'}]


## 7. Register Listeners

These handlers get called every time the Issuer agent receives a message. Specifically, whenever a connection transitions to the active state it issues a credential to that connection.

In [9]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))
        
        ## Offer credential
        loop = asyncio.get_event_loop()
        loop.create_task(agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


## 8. Create Multi Use invitation

Copy the invite across to the [Data Scientist](http://127.0.0.1:8888/notebooks/2%20-%20SSI_Syft_Data_Scientist.ipynb) notebook. Use this to establish a connection between the data scientist and the issuer.

In [10]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(multi_use="true")
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite["invitation"])


Connection message {'state': 'invitation', 'routing_state': 'none', 'connection_id': 'bd621565-1995-452b-9eb1-f9acba8c9c22', 'initiator': 'self', 'invitation_mode': 'multi', 'created_at': '2020-12-28 15:56:58.104902Z', 'accept': 'auto', 'updated_at': '2020-12-28 15:56:58.104902Z', 'invitation_key': 'HPC7JC5TRAEQjGy5Nn5fkzb835PC7dkkjHwcDX4RPuqY'} bd621565-1995-452b-9eb1-f9acba8c9c22
Connection ID bd621565-1995-452b-9eb1-f9acba8c9c22
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1e140337-4542-4e37-aa0a-bfc50baa80f5', 'serviceEndpoint': 'https://1589988d2293.ngrok.io', 'label': 'ISSUER', 'recipientKeys': ['HPC7JC5TRAEQjGy5Nn5fkzb835PC7dkkjHwcDX4RPuqY']}
{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "1e140337-4542-4e37-aa0a-bfc50baa80f5", "serviceEndpoint": "https://1589988d2293.ngrok.io", "label": "ISSUER", "recipientKeys": ["HPC7JC5TRAEQjGy5Nn5fkzb835PC7dkkjHwcDX4RPuqY"]}
Connection message {'state': 'i

Handle Credentials
Credential exchange b880be24-ab0c-48ea-8bcb-545994bf8899, role: issuer, state: offer_sent
Offering: [{'name': 'connection_permitted', 'value': '1'}]
Handle Credentials
Credential exchange b880be24-ab0c-48ea-8bcb-545994bf8899, role: issuer, state: request_received
Offering: [{'name': 'connection_permitted', 'value': '1'}]
Handle Credentials
Credential exchange b880be24-ab0c-48ea-8bcb-545994bf8899, role: issuer, state: credential_issued
Offering: [{'name': 'connection_permitted', 'value': '1'}]
Handle Credentials
Credential exchange b880be24-ab0c-48ea-8bcb-545994bf8899, role: issuer, state: credential_acked
Offering: [{'name': 'connection_permitted', 'value': '1'}]
Connection message {'state': 'invitation', 'routing_state': 'none', 'my_did': 'G7TPtWwTHZxp9gE6KdUYE2', 'connection_id': 'bf94e142-69c2-4370-8d14-8e2466185352', 'initiator': 'multiuse', 'invitation_mode': 'once', 'created_at': '2020-12-28 16:36:18.169683Z', 'accept': 'auto', 'updated_at': '2020-12-28 16:36:1

In [13]:
response = await agent_controller.connections.get_connection(connection_id)
print(response)

{'routing_state': 'none', 'accept': 'auto', 'created_at': '2020-12-19 16:59:04.665412Z', 'invitation_key': 'EpPcMjU8sWYSS5ivVYZJ4vLF9ZPYaVi7YhAPQCLS5gTf', 'connection_id': '8ec72a34-c2d8-4a3b-ada4-38473729693c', 'initiator': 'self', 'invitation_mode': 'multi', 'state': 'invitation', 'updated_at': '2020-12-19 16:59:04.665412Z'}


In [14]:
response = await agent_controller.terminate()
print(response)

None
